# Project Aromaloka

# Code

## Library

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import requests


## Request Data From API

In [ ]:
# Make API request
response_perfume = requests.get('https://aromaloka-api-muoaf7jkpa-et.a.run.app/perfumes/')

## Data Preprocessing

In [ ]:
# Check if the request was successful
if response_perfume.status_code == 200:
    # Extract the data from the response
    api_data = response_perfume.json()

    # Convert the data to a pandas DataFrame
    data = pd.DataFrame(api_data)

    # Sort the DataFrame by the 'id' column
    data['id'] = data['id'].astype(int)
    data = data.sort_values('id').reset_index(drop=True)
else:
    print('API request failed with status code:', response.status_code)

## Feature Engineering

In [ ]:
fragrance_notes = data['top_notes1'] + ' ' + data['top_notes2'] + ' ' + data['top_notes3'] + ' ' + data['mid_notes1'] + ' ' + data['mid_notes2'] + ' ' + data['mid_notes3'] + ' ' + data['base_notes1'] + ' ' + data['base_notes2'] + ' ' + data['base_notes3']
data['fragrance_notes'] = fragrance_notes

# Text Preprocessing
data['fragrance_notes'] = data['fragrance_notes'].str.lower()
data['fragrance_notes'] = data['fragrance_notes'].str.replace('[^\w\s]', '')
data['fragrance_notes'] = data['fragrance_notes'].str.split()

# One-Hot Encoding
one_hot_encoded = pd.get_dummies(data[['concentration', 'gender']])
data = pd.concat([data, one_hot_encoded], axis=1)

<ipython-input-4-d2c82b7ad91a>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  data['fragrance_notes'] = data['fragrance_notes'].str.replace('[^\w\s]', '')


## Create Input Data

In [ ]:
# Step 3: Create Input Data
count_vectorizer = CountVectorizer()
fragrance_matrix = count_vectorizer.fit_transform(data['fragrance_notes'].apply(' '.join))

# Normalize Fragrance Matrix
fragrance_matrix = fragrance_matrix.toarray().astype(np.float32)
fragrance_matrix /= np.linalg.norm(fragrance_matrix, axis=1, keepdims=True)

## Build Model

In [ ]:
# Step 4: Build Model
perfume_input = Input(shape=(fragrance_matrix.shape[1],), name='perfume_input')
concentration_input = Input(shape=(len(one_hot_encoded.columns),), name='concentration_input')
gender_input = Input(shape=(len(one_hot_encoded.columns),), name='gender_input')

x = concatenate([perfume_input, concentration_input, gender_input])

x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
output = Dense(fragrance_matrix.shape[1], activation='softmax')(x)

model = Model(inputs=[perfume_input, concentration_input, gender_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy')

## Train Model

In [ ]:
# Step 5: Train Model
X_perfume = fragrance_matrix
X_concentration = one_hot_encoded.values
X_gender = one_hot_encoded.values

early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

model.fit([X_perfume, X_concentration, X_gender], X_perfume,
          batch_size=32,
          epochs=50,
          validation_split=0.2,
          callbacks=[early_stopping])

Epoch 1/50
9/9 [==============================] - 1s 32ms/step - loss: 17.0590 - val_loss: 16.8059
Epoch 2/50
9/9 [==============================] - 0s 7ms/step - loss: 16.9633 - val_loss: 16.6878
Epoch 3/50
9/9 [==============================] - 0s 8ms/step - loss: 16.7872 - val_loss: 16.4450
Epoch 4/50
9/9 [==============================] - 0s 7ms/step - loss: 16.4409 - val_loss: 15.9699
Epoch 5/50
9/9 [==============================] - 0s 9ms/step - loss: 15.8227 - val_loss: 15.2042
Epoch 6/50
9/9 [==============================] - 0s 9ms/step - loss: 14.9969 - val_loss: 14.4148
Epoch 7/50
9/9 [==============================] - 0s 12ms/step - loss: 14.4089 - val_loss: 14.2577
Epoch 8/50
9/9 [==============================] - 0s 7ms/step - loss: 14.7238 - val_loss: 15.0892
Epoch 9/50
9/9 [==============================] - 0s 9ms/step - loss: 16.1472 - val_loss: 17.0289
Epoch 10/50
9/9 [==============================] - 0s 7ms/step - loss: 18.4476 - val_loss: 19.6613
Epoch 11/50
9/9 [

## Save Model

In [ ]:
# Save the trained model
model.save('perfume_recommendation_model.h5')

from google.colab import files

# Specify the path of the saved model file
model_path = 'perfume_recommendation_model.h5'

# Download the model file
files.download(model_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Generate Recommendation

In [ ]:
# Step 6: Generate Recommendations
perfume_features_model = Model(inputs=[perfume_input, concentration_input, gender_input], outputs=x)
perfume_features = perfume_features_model.predict([X_perfume, X_concentration, X_gender])

def get_recommendations(perfume_ids, top_n=5):
    all_recommendations = []

    for perfume_id in perfume_ids:
        perfume_index = data[data['id'] == perfume_id].index
        if len(perfume_index) > 0:
            perfume_index = perfume_index[0]
            similarity_scores_per_perfume = cosine_similarity([perfume_features[perfume_index]], perfume_features)[0]
            similar_perfume_indices = np.argsort(-similarity_scores_per_perfume)[1:top_n+1]
            similar_perfumes = data.loc[similar_perfume_indices, 'id']
            # similarity_scores = similarity_scores_per_perfume[similar_perfume_indices]
            all_recommendations.extend(similar_perfumes.values)

    unique_recommendations = list(set(all_recommendations))
    return unique_recommendations[:top_n]

11/11 [==============================] - 0s 2ms/step


# Example of Use

In [ ]:
# Example usage
perfume_ids = [6, 1, 284]  # Specify the perfume IDs for which you want recommendations
recommendations = get_recommendations(perfume_ids)
print("=============== Perfume Input ===============")
for id in perfume_ids:
  matching_rows = data.loc[data['id'] == id]
  if len(matching_rows) > 0:
      perfume_brand = matching_rows['brand'].values[0]
      perfume_variant = matching_rows['variant'].values[0]
      perfume_gender = matching_rows['gender'].values[0]
      fragrance_notes = ','.join(matching_rows['fragrance_notes'].values[0])
      print("Perfume ID:", id)
      print("Perfume Brand:", perfume_brand)
      print("Perfume Variant:", perfume_variant)
      print("Perfume Gender:", perfume_gender)
      print("Fragrance Notes:", fragrance_notes)
      print()

print("=============== Recommendation (Top 5) ===============")
for recommendation in recommendations:
    matching_rows = data.loc[data['id'] == recommendation]
    if len(matching_rows) > 0:
        perfume_brand = matching_rows['brand'].values[0]
        perfume_variant = matching_rows['variant'].values[0]
        perfume_gender = matching_rows['gender'].values[0]
        fragrance_notes = ','.join(matching_rows['fragrance_notes'].values[0])
        similarity_score = cosine_similarity([perfume_features[data[data['id'] == recommendation].index[0]]], perfume_features)[0][0]
        print("Perfume ID:", recommendation)
        print("Perfume Brand:", perfume_brand)
        print("Perfume Variant:", perfume_variant)
        print("Perfume Gender:", perfume_gender)
        print("Similarity Score:", similarity_score)
        print("Fragrance Notes:", fragrance_notes)
        print()
    else:
        print("No matching data found for perfume ID:", recommendation)

=============== Perfume Input ===============
Perfume ID: 6
Perfume Brand: Alien Objects
Perfume Variant: XOXO
Perfume Gender: Feminime
Fragrance Notes: dew,strawberry,lotus,tulip,musk,oakmoss,orange,blossom

Perfume ID: 1
Perfume Brand: Alchemist
Perfume Variant: Powder Room
Perfume Gender: Unisex
Fragrance Notes: watery,red,rose,pink,pepper,violet,orange,blossom,peony,musk,marine,sandalwood

Perfume ID: 284
Perfume Brand: S.TOI Fragrances
Perfume Variant: Indigo
Perfume Gender: Masculine
Fragrance Notes: calabrian,bergamot,grapefruit,hazelnut,cedarwood,honey,spices,amber,oakmoss,woody

=============== Recommendation (Top 5) ===============
Perfume ID: 225
Perfume Brand: OUDS
Perfume Variant: PSYCH
Perfume Gender: Unisex
Similarity Score: 0.9994128
Fragrance Notes: aldehyde,bergamot,lemon,apple,violet,pineapple,ambergris,musk,cedarwood

Perfume ID: 131
Perfume Brand: Joie the Lab
Perfume Variant: Memoir
Perfume Gender: Masculine
Similarity Score: 0.993135
Fragrance Notes: calabrian,be